### Задание по программированию: Рекомендательные системы

### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

    Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
    Если товар не встречался в обучающей выборке, его популярность равна 0.
    Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
    Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

### Задание

    На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
    Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

    Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
    Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [1]:
# import things
from collections import Counter, OrderedDict
import numpy as np
import pandas as pd

In [2]:
#create counters for frequence dictionaries
view_counter = Counter()
purchase_counter = Counter()

with open('train.txt', 'r') as f:                            # open file
    for line in f.readlines():                               # read line
        views, purchases = line.strip().split(';')           # split to views and purchases
        for view in views.split(','):                        # split into individual viewed items
            view_counter[view] += 1                          # add it to view_counter dictionary
        if purchases != '':                                  # check if there were a purchase
            for purchase in purchases.split(','):            # split into individual purchased items
                purchase_counter[purchase] += 1              # add item to purchase_counter dictionary



In [3]:
def calculate_metrics(is_view, file):
    '''
    is_view - selection for different calculations: for viewed item, or purchased
    file - filepath to session data
    '''
    if is_view == True:
        dictionary = view_counter
    else:
        dictionary = purchase_counter
    k = 5   # lenght of k
    with open(file, 'r') as f:     # open datafile
        AR = np.zeros(k)           # empty array for RECALL sum
        AP = np.zeros(k)           # empty array for PRECISION sum
        sessions_count = 0         # session counter to calculate average values
        for line in f.readlines():
            views, purchases = line.strip().split(';')
            if purchases != '':
                views = views.split(',')
                purchases = set(purchases.split(','))     # remove doubles from purchases
                rec = sorted(OrderedDict.fromkeys(views), key=lambda x: dictionary.get(x, 0),reverse=True)[:5]# sorted by dictionaries list
                rec_hits = np.array(list(map(lambda x: x in purchases, rec))) #intersection between purchases and sorted list
                matches = np.zeros(k)                  
                matches[:len(rec_hits)] = rec_hits     #number of matches between two lists
                sessions_count += 1
                AR +=  np.cumsum(matches) / len(purchases)
                AP += np.cumsum(matches) / (np.arange(k) + 1)

    for k in range(1, 6, 4):
        print('AR@{}: {}'.format(k,[round(x, 2) for x in AR / sessions_count][k-1]))
        print('AP@{}: {}'.format(k,[round(x, 2) for x in AP / sessions_count][k-1]))

### AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5

In [4]:
# views_popularity_train
calculate_metrics(True, 'train.txt')

AR@1: 0.44
AP@1: 0.51
AR@5: 0.82
AP@5: 0.21


In [5]:
# views_popularity_test
calculate_metrics(True, 'test.txt')

AR@1: 0.42
AP@1: 0.48
AR@5: 0.8
AP@5: 0.2


In [6]:
# purchases_popularity_train
calculate_metrics(False, 'train.txt')

AR@1: 0.69
AP@1: 0.8
AR@5: 0.93
AP@5: 0.25


In [7]:
# purchases_popularity_test
calculate_metrics(False, 'test.txt')

AR@1: 0.46
AP@1: 0.53
AR@5: 0.82
AP@5: 0.21
